In [ ]:
#|default_exp core

# Claudio source code

This is the 'literate' source code for Claudio. You can view the fully rendered version of the notebook [here](https://answerdotai.github.io/claudio/core.html), or you can clone the git repo and run the [interactive notebook](https://github.com/AnswerDotAI/claudio/blob/main/00_core.ipynb) in Jupyter. The notebook is converted the [Python module claudio/core.py](https://github.com/AnswerDotAI/claudio/blob/main/claudio/core.py) using [nbdev](https://nbdev.fast.ai/). The goal of this source code is to both create the Python module, and also to teach the reader *how* it is created, without assuming much existing knowledge about Claude's API.

## Setup

In [ ]:
import os
# os.environ['ANTHROPIC_LOG'] = 'debug'

To print every HTTP request and response in full, uncomment the above line. This functionality is provided by Anthropic's SDK.

In [ ]:
#| export
import tokenize, ast, inspect, inspect, typing
import xml.etree.ElementTree as ET, json
from collections import abc

from anthropic import Anthropic
from anthropic.types import Usage, TextBlock, Message
from anthropic.types.beta.tools import ToolsBetaMessage, tool_use_block
from inspect import Parameter
from io import BytesIO
try: from IPython.display import Markdown,HTML
except: Markdown,HTML=None,None

from fastcore.docments import docments
from fastcore.utils import *

In [ ]:
#| exports
models = 'claude-3-opus-20240229','claude-3-sonnet-20240229','claude-3-haiku-20240307'

These are the current versions of Anthropic's model at the time of writing.

In [ ]:
#| export
empty = Parameter.empty

In [ ]:
model = models[-1]

For examples, we'll use Haiku, since it's fast and cheap (and surprisingly good!)

## Antropic SDK

In [ ]:
cli = Anthropic()

This is what Anthropic's SDK provides for interacting with Python. To use it, pass it a list of *messages*, with *content* and a *role*. The roles should alternate between *user* and *assistant*.

In [ ]:
m = {'role': 'user', 'content': "I'm Jeremy"}
r = cli.messages.create(messages=[m], model=model, max_tokens=100)
r

Message(id='msg_014HjqFL5CM3dZ1z98nnh2f6', content=[TextBlock(text="It's nice to meet you Jeremy! I'm Claude, an AI assistant created by Anthropic. I'm here to help out however I can. Please let me know if you have any questions or if there is anything I can assist you with.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=10, output_tokens=54))

Now we're going to write our first function which will be included to the `claudia/core.py` module. In the rendered version of the notebook, for each exported function you'll see 3 things, in this order:

- The source code
- The signature
- The doc string.

After that, we'll generally provide a bit more detail on what we've created, and why, along with a sample usage.

In [ ]:
#| exports
def mk_msg(content, role='user', **kw):
    "Helper to create a `dict` appropriate for a Claude message"
    if hasattr(content, 'content'): content,role = content.content,content.role
    if isinstance(content, abc.Mapping): content=content['content']
    return dict(role=role, content=content, **kw)

We make things a bit more convenient by writing a function to create these messages for us.

As you see from the source, `mk_msg` doesn't only handle `str` for `content`, but can also deal with a `dict` or an object containing `content` (such as for from assistant responses).

In [ ]:
prompt = "I'm Jeremy"
m = mk_msg(prompt)
m

{'role': 'user', 'content': "I'm Jeremy"}

In [ ]:
r = cli.messages.create(messages=[m], model=model, max_tokens=100)
r

Message(id='msg_01JW2U8pDjrbPPqoq9rjqzNv', content=[TextBlock(text="It's nice to meet you Jeremy! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know how I can be of assistance.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=10, output_tokens=47))

In [ ]:
#| exports
def find_block(r, blk_type=TextBlock):
    "Find the first block of type `blk_type` in `r.content`"
    return first(o for o in r.content if isinstance(o,blk_type))

This makes it easier to grab the needed parts of Claude's responses, which can include multiple pieces of content. By default, we look for the first text block.

In [ ]:
find_block(r)

TextBlock(text="It's nice to meet you Jeremy! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know how I can be of assistance.", type='text')

In [ ]:
#| exports
def contents(r):
    "Helper to get the contents from Claude response `r`"
    blk = find_block(r)
    if not blk: blk = r.content[0]
    return blk.text.strip() if hasattr(blk,'text') else blk

For display purposes, we often just want to show the text itself.

In [ ]:
contents(r)

"It's nice to meet you Jeremy! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know how I can be of assistance."

In [ ]:
#| exports
def mk_msgs(msgs, **kw):
    "Helper to set 'assistant' role on alternate messages"
    if isinstance(msgs,str): msgs=[msgs]
    return [mk_msg(o, ('user','assistant')[i%2], **kw) for i,o in enumerate(msgs)]

LLMs, including Claude, don't actually have state, but instead dialogs are created by passing back all previous prompts and responses every time. With Claude, they always alternate *user* and *assistant*. Therefore we create a function to make it easier to build up these dialog lists.

In [ ]:
msgs = mk_msgs([prompt, r, 'I forgot my name. Can you remind me please?'])
msgs

[{'role': 'user', 'content': "I'm Jeremy"},
 {'role': 'assistant',
  'content': [TextBlock(text="It's nice to meet you Jeremy! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know how I can be of assistance.", type='text')]},
 {'role': 'user', 'content': 'I forgot my name. Can you remind me please?'}]

Now, if we pass this list of messages to Claude, the model treats it as a conversation to respond to.

In [ ]:
r = cli.messages.create(messages=msgs, model=model, max_tokens=200)
r

Message(id='msg_01QU9shSR6KX5knJgzq7rDxf', content=[TextBlock(text="I do not actually know your name - you said earlier that your name is Jeremy, but if you've now forgotten that, I don't have any other information about your name. As an AI assistant, I don't have access to personal details about you unless you provide them to me directly. If you're unsure of your own name, I'd suggest trying to recall it or referring to other sources that may have that information.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=71, output_tokens=90))

In [ ]:
@patch
def _repr_html_(self:(ToolsBetaMessage,Message)):
    det = '</li>\n<li>'.join(f'{k}: {v}' for k,v in self.dict().items())
    return f"""{contents(self)}
<details><ul><li>
{det}
</ul></li></details>"""

Jupyter looks for a `_repr_html_` method in displayed objects; we add this in order to display just the content text, and collapse full details into a hideable section. Note that `patch` is from `fastcore`, and is used to add (or replace) functionality in an existing class.

In [ ]:
r

Message(id='msg_01QU9shSR6KX5knJgzq7rDxf', content=[TextBlock(text="I do not actually know your name - you said earlier that your name is Jeremy, but if you've now forgotten that, I don't have any other information about your name. As an AI assistant, I don't have access to personal details about you unless you provide them to me directly. If you're unsure of your own name, I'd suggest trying to recall it or referring to other sources that may have that information.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=71, output_tokens=90))

One key part of the response is the `usage` key, which tells us how many tokens we used:

In [ ]:
r.usage

Usage(input_tokens=71, output_tokens=90)

In [ ]:
#| exports
def usage(inp=0, out=0):
    "Slightly more concise version of `Usage`"
    return Usage(input_tokens=inp, output_tokens=out)

The constructor provided by Anthropic is rather verbose, so we clean it up a bit, using a lowercase version of the name.

In [ ]:
usage(5)

Usage(input_tokens=5, output_tokens=0)

In [ ]:
#| exports
@patch(as_prop=True)
def total(self:Usage): return self.input_tokens+self.output_tokens

Adding a `total` property to `Usage` makes it easier to see how many tokens we've used up altogether.

In [ ]:
usage(5,1).total

6

In [ ]:
#| exports
@patch
def __repr__(self:Usage): return f'In: {self.input_tokens}; Out: {self.output_tokens}; Total: {self.total}'

In python, patching `__repr__` let's us change how an object is displayed.

In [ ]:
r.usage

In: 71; Out: 90; Total: 161

In [ ]:
#| exports
@patch
def __add__(self:Usage, b):
    return usage(self.input_tokens+b.input_tokens, self.output_tokens+b.output_tokens)

And, patching `__add__` let's make `+` work on a class.

In [ ]:
r.usage+r.usage

In: 142; Out: 180; Total: 322

## Client

In [ ]:
#| exports
class Client:
    def __init__(self, model, cli=None):
        "Basic Anthropic messages client"
        self.model,self.use = model,Usage(input_tokens=0,output_tokens=0)
        self.c = (cli or Anthropic())

We'll create a simple `Client` for `Anthropic` which tracks usage stores the model to use. We don't add any methods right away -- instead we'll use `patch` for that so we can add and document them incrementally.

In [ ]:
c = Client(model)
c.use

In: 0; Out: 0; Total: 0

In [ ]:
#| exports
@patch
def _r(self:Client, r:ToolsBetaMessage):
    "Store the result of the message and accrue total usage"
    self.result = r
    self.use += r.usage
    return r

We use a `_` prefix on private methods, but we document them here in the interests of literate source code.

`_r` will be used each time we get a new result, to track usage and also to keep the result available for later.

In [ ]:
c._r(r)
c.use

In: 71; Out: 90; Total: 161

In [ ]:
#| exports
@patch
def __call__(self:Client, msgs, sp='', temp=0, maxtok=4096, stop=None, **kw):
    "Make a call to Claude without streaming"
    r = self.c.beta.tools.messages.create(
        model=self.model, messages=mk_msgs(msgs), max_tokens=maxtok, system=sp, temperature=temp, stop_sequences=stop, **kw)
    return self._r(r)

Defining `__call__` let's us use an object like a function (i.e it's *callable*). We use it as a small wrapper over `messages.create`.

In [ ]:
c('Hi')

ToolsBetaMessage(id='msg_01XKoCB5Zg82ntkA6CHQ5Gve', content=[TextBlock(text='Hello! How can I assist you today?', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=In: 8; Out: 12; Total: 20)

In [ ]:
c.use

In: 79; Out: 102; Total: 181

In [ ]:
#| exports
@patch
def stream(self:Client, msgs, sp='', temp=0, maxtok=4096, stop=None, **kw):
    "Make a call to Claude, streaming the result"
    with self.c.messages.stream(model=self.model, messages=mk_msgs(msgs), max_tokens=maxtok,
                                system=sp, temperature=temp, stop_sequences=stop, **kw) as s:
        yield from s.text_stream
        return self._r(s.get_final_message())

We also define a wrapper over `messages.stream`, which is like `messages.create`, but streams the response back incrementally.

In [ ]:
for o in c.stream('Hi'): print(o, end='')

Hello! How can I assist you today?

In [ ]:
c.use

In: 87; Out: 114; Total: 201

## Tool use

[Tool use](https://docs.anthropic.com/claude/docs/tool-use) lets Claude use external tools. The API uses JSON schema, so we need a way to map Python functions to JSON.

We'll use [docments](https://fastcore.fast.ai/docments.html) to make defining Python functions as ergonomic as possible. Each parameter (and the return value) should have a type, and a docments comment with the description of what it is. Here's an example:

In [ ]:
def silly_sum(
    a:int, # First thing to sum
    b:int=1, # Second thing to sum
    c:list[int]=None, # A pointless argument
) -> int: # The sum of the inputs
    "Adds a + b"
    return a + b

This is what `docments` makes of that:

In [ ]:
d = docments(silly_sum, full=True)
d

```json
{ 'a': { 'anno': <class 'int'>,
         'default': <class 'inspect._empty'>,
         'docment': 'First thing to sum'},
  'b': {'anno': <class 'int'>, 'default': 1, 'docment': 'Second thing to sum'},
  'c': {'anno': list[int], 'default': None, 'docment': 'A pointless argument'},
  'return': { 'anno': <class 'int'>,
              'default': <class 'inspect._empty'>,
              'docment': 'The sum of the inputs'}}
```

Note that this is an [AttrDict](https://fastcore.fast.ai/basics.html#attrdict) so we can treat it like an object, *or* a dict:

In [ ]:
d.a.docment, d['a']['anno']

('First thing to sum', int)

In [ ]:
#| exports
def _types(t:type)->tuple[str,Optional[str]]:
    "Tuple of json schema type name and (if appropriate) array item name"
    tmap = {int:"integer", float:"number", str:"string", bool:"boolean", list:"array", dict:"object"}
    if getattr(t, '__origin__', None) in  (list,tuple): return "array", tmap.get(t.__args__[0], "object")
    else: return tmap.get(t, "object"), None

This internal function is needed to convert Python types into JSON schema types.

In [ ]:
_types(list[int]), _types(int)

(('array', 'integer'), ('integer', None))

In [ ]:
#| exports
def _param(name, info):
    "json schema parameter given `name` and `info` from docments full dict"
    paramt,itemt = _types(info.anno)
    pschema = dict(type=paramt, description=info.docment)
    if itemt: pschema["items"] = {"type": itemt}
    if info.default is not empty: pschema["default"] = info.default
    return pschema

This private function converts a key/value pair from the `docments` structure into the `dict` that will be needed for the schema.

In [ ]:
n,o = first(d.items())
print(n,'//', o)
_param(n, o)

a // {'docment': 'First thing to sum', 'anno': <class 'int'>, 'default': <class 'inspect._empty'>}


{'type': 'integer', 'description': 'First thing to sum'}

In [ ]:
#| exports
def get_schema(f:callable)->dict:
    "Convert function `f` into a JSON schema `dict` for tool use"
    d = docments(f, full=True)
    ret = d.pop('return')
    paramd = {
        'type': "object",
        'properties': {n:_param(n,o) for n,o in d.items()},
        'required': [n for n,o in d.items() if o.default is empty]
    }
    desc = f.__doc__
    if ret.anno is not empty: desc += f'\n\nReturns:\n- type: {_types(ret.anno)[0]}'
    if ret.docment: desc += f'\n- description: {ret.docment}'
    return dict(name=f.__name__, description=desc, input_schema=paramd)

Putting this all together, we can now test getting a schema from `silly_sum`. The tool use spec doesn't support return annotations directly, so we put that in the description instead.

In [ ]:
s = get_schema(silly_sum)
desc = s.pop('description')
print(desc)
s

Adds a + b

Returns:
- type: integer
- description: The sum of the inputs


{'name': 'silly_sum',
 'input_schema': {'type': 'object',
  'properties': {'a': {'type': 'integer', 'description': 'First thing to sum'},
   'b': {'type': 'integer',
    'description': 'Second thing to sum',
    'default': 1},
   'c': {'type': 'array',
    'description': 'A pointless argument',
    'items': {'type': 'integer'},
    'default': None}},
  'required': ['a']}}

We'll now get ready to test this out -- first we need a function that Claude can call; we'll write a simple function that adds numbers together:

In [ ]:
def sums(
    # First thing to sum
    a:int,
    # Second thing to sum
    b:int=1
# The sum of the inputs
) -> int:
    "Adds a + b"
    return a + b

In [ ]:
a,b = 604542,6458932
pr = f"What is {a}+{b}?"
sp = "You must use the `sums` function instead of adding yourself, but don't mention what tools you use."
tools=[get_schema(sums)]

We'll start a dialog with Claude now. We'll store the messages of our dialog in `msgs`. The first message will be our prompt `pr`, and we'll pass our `tools` schema.

In [ ]:
msgs = mk_msgs(pr)
r = c(msgs, sp=sp, tools=tools)
r

ToolsBetaMessage(id='msg_01CNbyP4QHZcmrz315nEFHJc', content=[ToolUseBlock(id='toolu_01GZCfB1mAG58UMgv2Vosv9r', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 414; Out: 72; Total: 486)

When Claude decides that it should use a tool, it passes back a `ToolUseBlock` with the name of the tool to call, and the params to use.

We need to append the response to the dialog so Claude knows what's happening (since it's stateless).

In [ ]:
msgs.append(mk_msg(r))

We don't want to allow it to call just any possible function (that would be a security disaster!) so we create a *namespace* -- that is, a dictionary of allowable function names to call.

In [ ]:
#| exports
def mk_ns(*funcs:list[callable]) -> dict[str,callable]:
    "Create a `dict` of name to function in `funcs`, to use as a namespace"
    return {f.__name__:f for f in funcs}

In [ ]:
ns = mk_ns(sums)
ns

{'sums': <function __main__.sums(a: int, b: int = 1) -> int>}

In [ ]:
#| exports
def call_func(tr, ns=None):
    "Call the function in the tool response `tr`, using namespace `ns`"
    if ns is None: ns=globals()
    if not isinstance(ns, abc.Mapping): ns = mk_ns(*ns)
    fc = find_block(r, tool_use_block.ToolUseBlock)
    return ns[fc.name](**fc.input)

We can now use the function requested by Claude. We look it up in `ns`, and pass in the provided parameters.

In [ ]:
res = call_func(r, ns=ns)
res

7063474

In [ ]:
#| exports
def mk_toolres(r, res=None, ns=None):
    "Create a `tool_result` message from response `r`"
    if not hasattr(r, 'content'): return r
    tool = first(o for o in r.content if isinstance(o,tool_use_block.ToolUseBlock))
    if not tool: return r
    if res is None: res = call_func(r, ns)
    tr = dict(type="tool_result", tool_use_id=tool.id, content=str(res))
    return mk_msg([tr])

In order to tell Claude the result of the tool call, we pass back a `tool_result` message, created by calling `call_func`.

In [ ]:
tr = mk_toolres(r, res=res, ns=ns)
tr

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_01GZCfB1mAG58UMgv2Vosv9r',
   'content': '7063474'}]}

We add this to our dialog, and now Claude has all the information it needs to answer our question.

In [ ]:
msgs.append(tr)
contents(c(msgs, sp=sp, tools=tools))

'The sum of 604542 and 6458932 is 7063474.'

## Chat

Rather than manually adding the responses to a dialog, we'll create a simple `Chat` class to do that for us, each time we make a request. We'll also store the system prompt and tools here, to avoid passing them every time.

In [ ]:
#| exports
class Chat:
    def __init__(self, model=None, cli=None, sp='', tools=None):
        "Anthropic chat client"
        assert model or cli
        self.c = (cli or Client(model))
        self.h,self.sp,self.tools = [],sp,tools

The class stores the `Client` that will provide the responses in `c`, and a history of messages in `h`.

In [ ]:
sp = "If asked to add things up, use the `sums` function instead of doing it yourself. Never mention what tools you use."
chat = Chat(model, sp=sp, tools=[sums])
chat.c.use, chat.h

(In: 0; Out: 0; Total: 0, [])

In [ ]:
@patch
def __call__(self:Chat, pr, temp=0, maxtok=4096, stop=None, ns=None, **kw):
    if ns is None: ns=self.tools
    self.h = mk_msgs(self.h + [mk_toolres(pr, ns=ns)])
    if self.tools: kw['tools'] = [get_schema(o) for o in self.tools]
    res = self.c(self.h, sp=self.sp, temp=temp, maxtok=maxtok, stop=stop, **kw)
    self.h.append(mk_msg(res, role='assistant'))
    return res

The `__call__` method just passes the request along to the `Client`, but rather than just passing in this one prompt, it appends it to the history and passes it all along. As a result, we now have state!

In [ ]:
chat("I'm Jeremy")
contents(chat("What's my name?"))

'Your name is Jeremy, as you told me earlier.'

We automagically get streamlined tool use as well:

In [ ]:
pr = f"What is {a}+{b}?"
pr

'What is 604542+6458932?'

In [ ]:
r = chat(pr)
r

ToolsBetaMessage(id='msg_01BuonV8LEsPgVGxQcnKtwek', content=[TextBlock(text='Okay, let me calculate that for you:', type='text'), ToolUseBlock(id='toolu_018q7WHBsxxyzLWPmR9THBHN', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 493; Out: 83; Total: 576)

In [ ]:
chat(r)

ToolsBetaMessage(id='msg_013dUcWHJhbDVRThcFomcjsa', content=[TextBlock(text='The sum of 604,542 and 6,458,932 is 7,063,474.', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=In: 590; Out: 28; Total: 618)

It should be correct, because it actually used our Python function to do the addition...

In [ ]:
a+b

7063474

## XML helpers

In [ ]:
#| exports
def hl_md(s, lang='xml'):
    "Syntax highlight `s` using `lang`"
    if Markdown: return Markdown(f'```{lang}\n{s}\n```')
    print(s)

In [ ]:
#| exports
def to_xml(node, hl=False):
    "Convert `node` to an XML string"
    def mk_el(tag, cs, attrs):
        el = ET.Element(tag, attrib=attrs)
        if isinstance(cs, list): el.extend([mk_el(*o) for o in cs])
        elif cs is not None: el.text = str(cs)
        return el

    root = mk_el(*node)
    ET.indent(root)
    res = ET.tostring(root, encoding='unicode')
    return hl_md(res) if hl else res

In [ ]:
#| exports
def xt(tag, c=None, **kw):
    "Helper to create appropriate data structure for `to_xml`"
    kw = {k.lstrip('_'):str(v) for k,v in kw.items()}
    return tag,c,kw

In [ ]:
#| exports
g = globals()
tags = 'div','img','h1','h2','h3','h4','h5','p','hr','span','html'
for o in tags: g[o] = partial(xt, o)

In [ ]:
a = html([
    p('This is a paragraph'),
    hr(),
    xt('x-custom', foo='bar'),
    img(src='http://example.prg'),
    div([
        h1('This is a header'),
        h2('This is a sub-header', style='k:v'),
    ], _class='foo')
])

In [ ]:
to_xml(a, True)

```xml
<html>
  <p>This is a paragraph</p>
  <hr />
  <x-custom foo="bar" />
  <img src="http://example.prg" />
  <div class="foo">
    <h1>This is a header</h1>
    <h2 style="k:v">This is a sub-header</h2>
  </div>
</html>
```

In [ ]:
#|export
def json_to_xml(d:dict, rnm:str)->str:
    "Convert `d` to XML with root name `rnm`"
    root = ET.Element(rnm)
    def build_xml(data, parent):
        if isinstance(data, dict):
            for key, value in data.items(): build_xml(value, ET.SubElement(parent, key))
        elif isinstance(data, list):
            for item in data: build_xml(item, ET.SubElement(parent, 'item'))
        else: parent.text = str(data)
    build_xml(d, root)
    ET.indent(root)
    return ET.tostring(root, encoding='unicode')

In [ ]:
a = dict(surname='Howard', firstnames=['Jeremy','Peter'],
         address=dict(state='Queensland',country='Australia'))
print(json_to_xml(a, 'person'))

<person>
  <surname>Howard</surname>
  <firstnames>
    <item>Jeremy</item>
    <item>Peter</item>
  </firstnames>
  <address>
    <state>Queensland</state>
    <country>Australia</country>
  </address>
</person>


## Images

Not done yet.

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()